### Relevant scripts and explanations of what they do (in order of how they're used):
* `transient_search.sh`: this script downloads data from the local BAT machine at Goddard. It searches for the overlap between e.g. a LIGO probability map and the BAT FoV. The input is as follows:
> `source_name: the source name. This will be the folder name in the result directory
T0: enter either UTC or Swift MET 
T0_type: enter the word "UTC" or "MET", depends on which T0 you entered.
src_RA: RA for an input source to be searched in the event data.
src_DEC: DEC for an input source to be searched in the event data
input_img: An input image to be searched whether it overlaps with BAT FOV (e.g., LIGO probability map). Enter "NONE" if unavailable.
input_ima_Phil: Phil's probability map (covolving the LIGO map with nearby galaxies). Enter "NONE" if unavailable.
Example:./transient_search.sh S190510g 2019-05-10T02:59:39.292 UTC 0.0 0.0 /local/data/bat2/batusers/batgroup/BAT_GW/results/S190510g/bayestar.fits.gz /local/data/bat2/batusers/batgroup/BAT_GW/results/S190510g/bayestar_convolved.fits.gz`

This script relies on `battools` and `batcaldb` from the `headas` package. It also requires `kapteyn` to run---but this is not needed for the analysis (it's needed for Amy's analysis.).

This script results in an *info file* and *BAT lightcurves* (64 ms, 1 s, and 1.6 s) in four energy channels (15–25 keV, 25–50 keV, 50–100 keV, and 100–350 keV) generated from the central region of the BAT field of view. An example of the info file is pasted below: 

> `## This is a temporary file to store intermediate info of the script
pass_data_dir: /local/data/bat2/pass_data/2020
att_folder: /local/data/bat2/transient_monitor/attitude
Swift_pointing_folder: /local/data/bat2/batusers/batgroup/BAT_GW/Swift_pointing
LIGO_notice_folder: /local/data/bat2/batusers/batgroup/BAT_GW/LIGO_notice
source_name: GW200112_155838
T0_input: 2020-01-12T15:58:38
T0_input_flag: UTC
T0: 600537542.25
Year: 2020
Month: 01
Day: 12
DOY: 012
time_UTC: 2020-01-12T15:58:38 UTC
pass_data_flag: 7
src_RA: 0.0
src_DEC: 0.0
search_time_window: 30.0
output_dir: /local/data/bat1/batusers/mcrnog/results/GW200112_155838
input_img: /local/data/bat1/batusers/mcrnog/results/O3b/IGWN-GWTC3p0-v1-GW200112_155838_PEDataRelease_cosmo_reweight_C01:Mixed.fits
input_img_Phil: NONE
healpy_flag: 0
source_select_flag: 0
Earth_RA: 105.927635
Earth_DEC: 18.619223
BAT_RA: 319.23399898337846
BAT_DEC: 20.0597162685828
BAT_ROLL: 213.1244354248047
flag_att: 1
input_src_pcode: 0.377001783758
input_src_FOV_flag: 1
BAT_FOV_img: /local/data/bat1/batusers/mcrnog/results/GW200112_155838/batfov.fits
FOV_plot: bat_fov.png
flag_quad_rate: 1
flag_ms_rate: 1
flag_1s_rate: 1
Number_of_good_detector: 15396
`

WARNING: Note that BAT machine stores the data in distinct directories by day. Sometimes, the time interval you are searching for will be divided into two different directories. I checked for these manually and in few cases put the data together in the same directory and run the code again. This surely can be automated. 

* To run multiple searches at once, see `swift_searches.sh`.

* `calc_BAT_ul.py` is the Python script used to calculate upper limits from the input BAT light curve. It uses the response functions generated via NITRATES for different BAT IDs. The upper limit is calculated assuming the Band function with normal hardness and over 1 second (assumed duration of the short GRB). 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits as pyfits
import sys, os
import healpy as hp
from calc_BAT_ul import *

from copy import copy

In [3]:
events = ["GW200112_155838"] # here, you may input a list of GW events for which you have obtained the BAT lightcurves

In [4]:
for x in events:
    
    name = x # looping through the event names 
    
    bat_info = "test-GW200112_155838/info.txt" # change if you input a list
    lines = open(bat_info).readlines()
    bat_ra = float(lines[25].split()[-1]) # this is consistent throughout info files
    bat_dec = float(lines[26].split()[-1])
    earth_ra = float(lines[23].split()[-1])
    earth_dec = float(lines[24].split()[-1])
    earth_rad = 69. # Earth radius for Swift BAT
    t0 = float(lines[8].split()[-1]) # LIGO trigger time 
    
    # loading the 1-s BAT ligtcurve
    lc_path = "test-GW200112_155838/1s_rate_sort.lc" # this is the 1-second lightcurve
    lc_1s = pyfits.open(lc_path)
    counts_1s = lc_1s[1].data['COUNTS']
    time_1s = lc_1s[1].data['TIME'] - t0
    rate_1s = counts_1s 

    # starting the light curve from -1 to 30 to match GBM search window
    index = [i for i,x in enumerate(time_1s) if x>=-1 and x<=30] # defining your time window
    time_new = time_1s[index]
    counts_new = counts_1s[index]
    rate_new = rate_1s[index]
    std = np.std(rate_new) # std from the 1-second lightcurve
    
    Ndets_tot = 32768.0 # total dets 
    Ndets_active = float(lines[36].split()[-1]) # number of active detectors
    Ndet_ratio = Ndets_active / Ndets_tot

    IDs = [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33]
    rate_std = std # the std of rates from the LC
    rate_upper_limit = 5*rate_std
    # print("5sigma rate upper limit: ", rate_upper_limit)
    
    ul_5sigma = [] 
    
    for i in IDs:
        grid_id = i 
        # print("grid_id: ", grid_id)

        # using energy bin 15-350 and ignoring 350-500
        chan_low = 0 
        chan_hi = 3

        # getting the NITRATES DRM table
        drm_tab = get_drm_tab(grid_id)

        # response matrix using selected energy bins and corrected for number of active dets
        drm_matrix = drm_tab['MATRIX'][:,chan_low:(chan_hi+1)] * Ndet_ratio 

        # find the flux that gives an expected rate equal to the rate upper limit
        flux_upper_limit = rate2band_eflux(rate_upper_limit, drm_matrix,\
                                       drm_tab['ENERG_LO'], drm_tab['ENERG_HI'],\
                                       alpha, beta, Epeak, flux_elo, flux_ehi)
        #print("5-sigma flux upper limit [erg/cm2/s]: ", flux_upper_limit)
        ul_5sigma.append(flux_upper_limit)
        np.save(os.path.join('test-GW200112_155838', name + "_BAT_gridID_ul.npy"), ul_5sigma)
        

In [7]:
print(ul_5sigma)

[3.4624081206900016e-05, 2.1237481231121264e-05, 2.2501935420480577e-05, 2.1135194816322752e-05, 3.494110421536421e-05, 3.8797680982721596e-05, 1.5767093883516585e-05, 1.0197651912762587e-05, 1.0050553492933422e-05, 1.0262210214029397e-05, 1.5658979489515397e-05, 3.887152818689719e-05, 3.7018831183912466e-05, 1.3240534475833341e-05, 8.425512324750266e-06, 8.061144074773289e-06, 8.493993064202517e-06, 1.3253955561827329e-05, 3.648656867248806e-05, 4.192764788784255e-05, 1.8845230316964014e-05, 1.1800834069083298e-05, 1.0166295632503064e-05, 1.1638115938065236e-05, 1.8331183878661247e-05, 4.037646314469517e-05, 3.600769668002244e-05, 2.3643556064632898e-05, 1.9989475408942405e-05, 2.363229908755505e-05, 3.5414076222018155e-05]
